In [2]:
import time, os
import pandas as pd
from tqdm import tqdm
import requests
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.alert import Alert

- 7월: ***19,240건***
- 8월: ***20,007건***
- 9월(30): ***15,993건***
- 10월: ***18,726건***
- 11월(30): ***20,991건***
- 12월: ***17,010건***

In [3]:
# 제주 여행 검색량의 20%만 크롤링하기 위해 월별 갯수 저장
jul = 19240//5
aug = 20007//5
sept = 15993//5
oct = 18726//5
nov = 20991//5
dec = 17010//5

In [13]:
# 드라이버 지정
current = '/Users/shimsomin/Documents/GitHub/COVID19-Tourism'
chrome_path = current + '/driver/chromedriver'

options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument("enable-automation")
options.add_argument('headless') # 창 안 띄움 -> 더 빠름
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36") # Headless 탐지 막기위해 User-Agent 값을 바꿔줌.

options.add_argument("window-size=1920x1080") # 화면크기
options.add_argument("--no-sandbox")
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument('--disable-site-isolation-trials')

#Step 1. 크롬 웹브라우저 실행
driver = webdriver.Chrome(chrome_path, options=options)

query = "제주 여행"
start_date = "2020-12-01"
end_date = "2020-12-31"
max_link_cnt = dec
crawling_month = '12'

naver_df = pd.DataFrame(columns=['날짜', '네이버/티스토리', '관광 명소', '제목', '내용', 'url', '글쓴이'])
url = "https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate={}&endDate={}&keyword={}".format(start_date, end_date, query)


In [14]:
driver.get(url)
soup = BeautifulSoup(url, 'lxml')

# 1차 크롤링 (url, 제목) 시작
link_cnt = 0
btn_cnt = 1

# 진행바 while문에 사용하기
pbar = tqdm(total = max_link_cnt+1)
while link_cnt <= max_link_cnt:
    # 한 페이지 안의 총 기사 개수
    t_len = len(driver.find_elements_by_class_name("list_search_post"))

    for i in range(1, t_len+1):
        a = driver.find_element_by_css_selector("#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner".format(i))

        # url 수집
        url = a.get_attribute('href')

        # 제목 수집
        title = driver.find_element_by_css_selector("#content > section > div.area_list_search > div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner > strong > span".format(i)).text
        
        naver_df.loc[link_cnt] = ['nan', '네이버', query, title, 'nan', url, 'nan']

        link_cnt += 1
    # print(link_cnt)
    pbar.update(t_len)
    
    btn_cnt += 1
    # 다음 버튼 누르기
    if btn_cnt % 10 == 1:
        if btn_cnt > 11:
            btn = driver.find_element_by_css_selector("#content > section > div.pagination > a.button_next")
            btn.send_keys(Keys.ENTER)
        else:
            btn = driver.find_element_by_css_selector("#content > section > div.pagination > a")
            btn.send_keys(Keys.ENTER)
        driver.implicitly_wait(3)
    else:
        if btn_cnt > 11:
            if (btn_cnt % 10) == 0:
                btn = driver.find_element_by_css_selector("#content > section > div.pagination > span:nth-child(11) > a")
            else:
                btn = driver.find_element_by_css_selector("#content > section > div.pagination > span:nth-child({}) > a".format((btn_cnt%10)+1))
        else:
            btn = driver.find_element_by_css_selector("#content > section > div.pagination > span:nth-child({}) > a".format(btn_cnt))

        btn.send_keys(Keys.ENTER)
        driver.implicitly_wait(3)

pbar.close()

3409it [03:19, 17.05it/s]                          


In [15]:
naver_df.tail()

,날짜,네이버/티스토리,관광 명소,제목,내용,url,글쓴이
3404,nan,네이버,제주 여행,제주도의 작은 마을 및 동네 여행 카름투어를 아시나요?,nan,https://blog.naver.com/fromairplane/222189681797,nan
3405,nan,네이버,제주 여행,확진자 동선 2곳 추가공개(12월 4일) 및 82번 확진자·일행 1명 역학조사 완료,nan,https://blog.naver.com/happyjejudo/222163010277,nan
3406,nan,네이버,제주 여행,"서귀포, 분위기 좋은 카페 ""서홍정원"" & ""연과자점""",nan,https://blog.naver.com/psk8808/222181937525,nan
3407,nan,네이버,제주 여행,"[5박6일 제주태교여행] Day6_⑤ 제주공항 면세점, 샤넬, 페라가모 시계랑 선글라스",nan,https://blog.naver.com/lalalarc/222190707700,nan
3408,nan,네이버,제주 여행,제주 82번째 코로나19 확진자 발생… 인천 확진자 접촉자,nan,https://blog.naver.com/happyjejudo/222159607759,nan


In [16]:
from tqdm import tqdm

# 2차 크롤링 (블로그 제목, 포스팅 날짜, 글쓴이) 시작
for i in tqdm(range(len(naver_df))):
    driver.get(naver_df.loc[i]['url'])
    driver.switch_to.frame('mainFrame')
    overlays = ".se-component.se-text.se-l-default" # 내용 크롤링
    contents = driver.find_elements_by_css_selector(overlays)

    content_list = ""

    for content in contents:
        content_list = content_list + content.text

    # 글쓴이 수집
    author = driver.find_element_by_xpath("""//*[@id="nickNameArea"]""").text
    
    # date 수집
    try:
        date = driver.find_element_by_css_selector(".se_publishDate.pcol2").text
    except:
        date = driver.find_element_by_css_selector(".date.fil5.pcol2._postAddDate").text
    
    naver_df.loc[i][['날짜', '내용', '글쓴이']] = [date, content_list, author]

100%|██████████| 3409/3409 [1:46:50<00:00,  1.88s/it]  


In [17]:
len(naver_df[naver_df['글쓴이'] =='nan'])

0

In [18]:
save_dir = current+'/data'
naver_df.to_csv(save_dir + '/naver_2020_{}.csv'.format(crawling_month), index=False)